### This file contains the tests to examine and evaluate the functions.

Required libraries and modules need to be imported

In [12]:
from tensorflow.keras.datasets import mnist
import keras_tuner as kt
from kerastuner.engine.hyperparameters import HyperParameters
from tensorflow.keras import layers, models
import a1

# the data from the MNIST dataset should be loaded and preprocessed 
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255 # we want the pixel values to be between 0 and 1
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255  

C:\Users\amirs\AppData\Local\Temp\ipykernel_18404\2829997608.py:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.engine.hyperparameters import HyperParameters


here is the part where i train and evaluate the build_deep_nn function.

In [13]:
# now the function build_model for Kerastuner can be defined
def build_model(hp):
    num_hidden = hp.Int('num_hidden', 1, 3, default = 2) # range between 1 to 3 with the default set to 2
    dropout_rate = hp.Float('dropout_rate', 0.0, 0.9, default = 0)  # range between 0 to 0.9 with the default set to 0
    hidden_size = hp.Int('hidden_size', 32, 256, step = 32, default = 128) # range between 32 to 256 with 32 step size and default set to 128

    hidden_sizes = [hidden_size] * num_hidden # dinamically assigns the right value for the input parameters of build_deep_nn 
    dropout_rates = [dropout_rate] * num_hidden # function in each iteration

    model = a1.build_deep_nn(28, 28, 1, num_hidden, hidden_sizes, dropout_rates, 10, 'softmax') # the shape of each image is 28*28
                                                                                                # with one dimension, we have 10 classes
                                                                                                # so the output-shape should be 10
                                                                                                # softmax is a commonly used activation
                                                                                                # function for classification approaches
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) 
    
    return model

# now we can create Kerastuner BayesianOptimization instance
tuner = kt.BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    num_initial_points=2,
    overwrite=True
)

# this function searches for the best hyperparameters
tuner.search(train_images, train_labels, epochs=5, validation_split=0.2)

# printing out the best hyperparameters and model summary
best_hp = tuner.oracle.get_best_trials(1)[0].hyperparameters.values
best_model = tuner.get_best_models(1)[0]

print("Best Hyperparameters:", best_hp)
print("Best Model Summary:")
best_model.summary()


Trial 10 Complete [00h 00m 46s]
val_accuracy: 0.9763333201408386

Best val_accuracy So Far: 0.9770833253860474
Total elapsed time: 00h 07m 11s
Best Hyperparameters: {'num_hidden': 1, 'dropout_rate': 0.27559674121811384, 'hidden_size': 256}
Best Model Summary:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 256)               200960    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                2570      
                                                                 
Total params: 203530 (795.04 KB)
Trainable params: 203530 (7

time to train our model based on the best hyper parameters we mentioned in the model summary of the keras tuner.

In [24]:
# setting the hyper paremeters that we found
best_model = a1.build_deep_nn(28, 28, 1, 1, [256], [0.27559674121811384], 10, 'softmax')
# we compile the model before using it
best_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# time to train the model and feed the data
best_model.fit(train_images, train_labels, epochs=5, validation_split=0.2)

Epoch 1/5
1500/1500 [==============================] - 10s 6ms/step - loss: 0.2940 - accuracy: 0.9128 - val_loss: 0.1442 - val_accuracy: 0.9562
Epoch 2/5
1500/1500 [==============================] - 9s 6ms/step - loss: 0.1351 - accuracy: 0.9590 - val_loss: 0.1041 - val_accuracy: 0.9688
Epoch 3/5
1500/1500 [==============================] - 10s 6ms/step - loss: 0.0988 - accuracy: 0.9696 - val_loss: 0.0919 - val_accuracy: 0.9716
Epoch 4/5
1500/1500 [==============================] - 11s 7ms/step - loss: 0.0792 - accuracy: 0.9756 - val_loss: 0.0842 - val_accuracy: 0.9746
Epoch 5/5
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0646 - accuracy: 0.9798 - val_loss: 0.0774 - val_accuracy: 0.9778


here's the evaluation of the model on the test dataset

In [25]:
test_loss, test_accuracy = best_model.evaluate(test_images, test_labels)

print("Test Accuracy:", test_accuracy)

313/313 [==============================] - 1s 3ms/step - loss: 0.0721 - accuracy: 0.9793
Test Accuracy: 0.9793000221252441


### manual tests for the first two function

In [14]:
import numpy as np

Testing the first function in different scenarios.

In [15]:
image = np.array([[[250,   2,   2], [  0, 12,   2], [  0,   56, 255]], 
                [[  2,   2,   2], [250, 255, 255], [127, 127, 127]],
                [[  4,   4,   4], [6, 55, 6], [8, 8, 8]]])
darkness_threshold = 50
image_result = a1.image_statistics(image, darkness_threshold)
print(f"Image Resolution: {image_result['resolution']}")
print(f"Number of Dark Pixels in each Channel: {image_result['dark_pixels']}")

Image Resolution: (3, 3)
Number of Dark Pixels in each Channel: (6, 5, 6)


In [16]:
image = np.array([[[250,   2,   2], [  0, 12,   2], [  0,   56, 255]], 
                [[  2,   2,   2], [250, 255, 255], [127, 127, 127]],
                [[  4,   4,   4], [6, 55, 6], [8, 8, 8]],
                [[  2,   2,   2], [250, 255, 255], [127, 127, 127]]])

In [17]:
darkness_threshold = 3
image_result = a1.image_statistics(image, darkness_threshold)
print(f"Image Resolution: {image_result['resolution']}")
print(f"Number of Dark Pixels in each Channel: {image_result['dark_pixels']}")

Image Resolution: (4, 3)
Number of Dark Pixels in each Channel: (4, 3, 4)


Testing the second function in different scenarios.

In [18]:
image = np.array([[[250,   2,   2], [  0, 255,   2], [  0,   0, 255]], 
                [[  2,   2,   2], [250, 255, 255], [127, 127, 127]]])

In [19]:
a1.bounding_box(image, (0, 0), (1, 0))

array([[[250,   2,   2]],

       [[  2,   2,   2]]])

In [20]:
a1.bounding_box(image, (0, 0), (1, 1))

array([[[250,   2,   2],
        [  0, 255,   2]],

       [[  2,   2,   2],
        [250, 255, 255]]])

In [21]:
image = np.array([[[250,   2,   2], [  0, 12,   2], [  0,   56, 255]], 
                [[  2,   2,   2], [250, 255, 255], [127, 127, 127]],
                [[  4,   4,   4], [6, 55, 6], [8, 8, 8]],
                [[  2,   2,   2], [250, 255, 255], [127, 127, 127]]])

In [22]:
a1.bounding_box(image, (0, 0), (2, 1))

array([[[250,   2,   2],
        [  0,  12,   2]],

       [[  2,   2,   2],
        [250, 255, 255]],

       [[  4,   4,   4],
        [  6,  55,   6]]])

In [23]:
a1.bounding_box(image, (0, 0), (3, 1))

array([[[250,   2,   2],
        [  0,  12,   2]],

       [[  2,   2,   2],
        [250, 255, 255]],

       [[  4,   4,   4],
        [  6,  55,   6]],

       [[  2,   2,   2],
        [250, 255, 255]]])

In [11]:
a1.bounding_box(image, (0, 0), (3, 2))

array([[[250,   2,   2],
        [  0,  12,   2],
        [  0,  56, 255]],

       [[  2,   2,   2],
        [250, 255, 255],
        [127, 127, 127]],

       [[  4,   4,   4],
        [  6,  55,   6],
        [  8,   8,   8]],

       [[  2,   2,   2],
        [250, 255, 255],
        [127, 127, 127]]])